In [35]:
%pwd

'/Users/deeplatiyan/Desktop/projects/medical-ai-chatbot'

In [36]:

from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [37]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [38]:
from dotenv import load_dotenv
import os

load_dotenv()
hf_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [39]:
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="tiiuae/falcon-rw-1b",  # or any supported public model
    huggingfacehub_api_token=hf_token,
    model_kwargs={"temperature": 0.4, "max_new_tokens": 500}
)

/var/folders/1n/x245xykx2sx4q_wwsmp6qfn80000gn/T/ipykernel_53577/2126930069.py:3: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(


In [40]:
repo_id="google/flan-t5-small"  # simpler and known to work

In [41]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

response = rag_chain.invoke({"input": "What is Acromegaly and gigantism?"})
print(response["answer"])

NameError: name 'retriever' is not defined

In [42]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

NameError: name 'docsearch' is not defined

In [43]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name="medicalbot",
    embedding=embeddings
)

NameError: name 'embeddings' is not defined

In [44]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [45]:
from langchain_huggingface import HuggingFaceEmbeddings

In [46]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name="medicalbot",
    embedding=embeddings
)

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

NameError: name 'embeddings' is not defined

In [47]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name="medicalbot",
    embedding=embeddings
)

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

NameError: name 'embeddings' is not defined

In [48]:
# 1. Load .env
from dotenv import load_dotenv
load_dotenv()

# 2. Setup Embeddings (HuggingFace free)
from langchain_huggingface import HuggingFaceEmbeddings  # Use this instead of deprecated one
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3. Pinecone Setup
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medicalbot"

# 4. Connect to existing Pinecone index
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [49]:
response = rag_chain.invoke({"input": "What is Acromegaly and gigantism?"})
print(response["answer"])

NameError: name 'rag_chain' is not defined

In [50]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [2]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    index_name="medicalbot",
    embedding=embeddings
)

retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

NameError: name 'embeddings' is not defined

In [3]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

class LocalLLMWrapper:
    def __init__(self, pipe):
        self.pipe = pipe

    def invoke(self, prompt):
        return self.pipe(prompt, max_new_tokens=100)[0]["generated_text"]

local_llm = LocalLLMWrapper(pipe)

In [4]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

question_answer_chain = create_stuff_documents_chain(local_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class '__main__.LocalLLMWrapper'>